In [ ]:
subcases_pattern="_1"
reference_location="../.."
source_name="Her X-1"
osa_version='OSA11.0'
nscw=3
mosaic=0
chi2_limit=1.2
systematic_fraction=0.01

#  ISGRI verification with SPI reference

simultaneous observations allow easy comparisong

one should take care to have the same effective selection

normalization need to be fitted and reported

In [ ]:
%matplotlib notebook
#import matplotlib.pylab as plt


import numpy as np
import importlib
from astroquery.simbad import Simbad
from astropy import coordinates as coord
import os
import numpy as np
import glob


In [ ]:
reference_location

# if reference_location  not git or http may be a problem
# dependency injection for this, if can

In [ ]:

subcase=None

reference_spectrum = None

for subcase_dir in glob.glob(reference_location+"/subcases/*"+subcases_pattern+"*"):
    print("inspecting", subcase_dir)
    
    fns = [os.path.basename(fn) for fn in glob.glob(subcase_dir+"/*")]
    
    try:
        reference_spectrum = [fn for fn in fns if fn.startswith("spectrum")][0]
        scwlist = open(subcase_dir+"/ISGRI_scw_list.txt").read().split()       
        
        reference_rmf = sorted([fn for fn in fns if fn.startswith("rmf_")])[-1]
    except: continue
    
    try:
        ref_sources = np.genfromtxt(subcase_dir+"/ISGRI_sources.txt", delimiter=",", names=True)
    except:
        ref_sources = []
        
    #print(fns)
    
    reference_dir=subcase_dir
    
    print("found",reference_dir, reference_spectrum, len(scwlist),len(ref_sources), ref_sources)
        
    break
        
reference_spectrum, reference_rmf

In [ ]:
result_table = Simbad.query_object(source_name)
source_coord = coord.SkyCoord(result_table['RA'][0], result_table['DEC'][0], unit=("hourangle", "deg"))

result_table

In [ ]:
import random
if nscw >0:
    random.seed(0)
    scw_pick = random.sample([
                s+"."+"001"
                for s in list(sorted(set( scwlist  ))) 
                if s.endswith("0010")
            ],nscw)


    scw_list_str = ",".join(scw_pick)
else:
    scw_list_str=list(sorted(set( scwlist  )))

scw_list_str

In [ ]:
import oda_api.api
import importlib
importlib.reload(oda_api)

disp = oda_api.api.DispatcherAPI(host="http://cdcihn/staging-1.2/dispatcher")

disp.get_instrument_description("isgri")



In [ ]:
by_lt={}

if mosaic == 1:

    for c_emin in np.linspace(20,35,2):

        image = disp.get_product(instrument="isgri", 
                         product="isgri_image", 
                         product_type="Real", 
                         osa_version=osa_version,
                         E1_keV=np.round(c_emin),
                         E2_keV=80.0,
                         scw_list=scw_list_str)

        sources=image.dispatcher_catalog_1.table[image.dispatcher_catalog_1.table['src_names']==source_name]
        source=sources[np.argmax(sources['significance'])]
        source

        d=image.mosaic_image_0.data_unit[1].data


        img=np.array(d.data)

        m_bkg=img<10
        m_bkg&=img!=0

        img_std = np.std(img[m_bkg])

        img[np.array(img)>img_std*5]=img_std*5

        plt.imshow(img)

        by_lt['%.10lg'%c_emin]=dict(
            emin=c_emin,            
            imgstd=img_std,
        )
        by_lt['%.10lg'%c_emin].update(dict([(n, source[n]) for n in source.colnames]))

        
    for lt, c in by_lt.items():
        print(lt, c['significance'],c['significance']/c['imgstd'])

In [ ]:
spectrum = disp.get_product(instrument="isgri", 
                 product="isgri_spectrum", 
                 product_type="Real", 
                 osa_version=osa_version,
                 E1_keV=25.0,
                 E2_keV=80.0,
                 scw_list=scw_list_str)

In [ ]:
specprod=[l for l in spectrum._p_list if l.meta_data['src_name'] == source_name]    

spec_fn="/tmp/isgri_spectrum_{}.fits".format(source_name.replace(' ', '_'))
arf_fn="/tmp/isgri_arf_{}.fits".format(source_name.replace(' ', '_'))
rmf_fn="/tmp/isgri_rmf_{}.fits".format(source_name.replace(' ', '_'))

specprod[0].write_fits_file(spec_fn)
specprod[1].write_fits_file(arf_fn)
specprod[2].write_fits_file(rmf_fn)

In [ ]:

from IPython.display import Image
from IPython.display import display 
import xspec
import shutil



In [ ]:
from astropy.io import fits

f=fits.open(reference_dir+"/"+reference_spectrum)
f[2].header['RESPFILE'] = 'NONE'

#f[2].header['RESPFILE']

f.writeto("reference_spec.fits", overwrite=True)

f=fits.open(reference_dir+"/"+reference_rmf)
f.writeto("reference_rmf.fits", overwrite=True)

#    s_ref.response=reference_rmf

#    s_ref.response=reference_rmf
    

In [ ]:
reference_spec="reference_spec.fits"
reference_rmf="reference_rmf.fits"
isgri_spec=spec_fn
isgri_arf=arf_fn
isgri_rmf=rmf_fn